In [1]:
%pylab inline
import pickle
perfectPolicy = pickle.load(open("perfectPolicy.p", "rb"))
perfectPolicy
len(perfectPolicy)
import matplotlib.pyplot as plt


%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [2]:
randomPolicy = dict()
for board in perfectPolicy:
    l = len(perfectPolicy[board])
    randomPolicy[board] = numpy.ones(l) / l
    print(board, randomPolicy[board])


(0, 0, 0, 0, 0, 0, 0, 0, 0) [0.11111111 0.11111111 0.11111111 0.11111111 0.11111111 0.11111111
 0.11111111 0.11111111 0.11111111]
(1, 0, 0, 0, 0, 0, 0, 0, 0) [0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125]
(1, 2, 0, 0, 0, 0, 0, 0, 0) [0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714]
(1, 2, 1, 0, 0, 0, 0, 0, 0) [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
(1, 2, 1, 2, 0, 0, 0, 0, 0) [0.2 0.2 0.2 0.2 0.2]
(1, 2, 1, 2, 1, 0, 0, 0, 0) [0.25 0.25 0.25 0.25]
(1, 2, 1, 2, 1, 2, 0, 0, 0) [0.33333333 0.33333333 0.33333333]
(1, 2, 1, 2, 1, 2, 0, 1, 0) [0.5 0.5]
(1, 2, 1, 2, 1, 2, 2, 1, 0) [1.]
(1, 2, 1, 2, 1, 2, 0, 1, 2) [1.]
(1, 2, 1, 2, 1, 0, 2, 0, 0) [0.33333333 0.33333333 0.33333333]
(1, 2, 1, 2, 1, 1, 2, 0, 0) [0.5 0.5]
(1, 2, 1, 2, 1, 1, 2, 2, 0) [1.]
(1, 2, 1, 2, 1, 1, 2, 0, 2) [1.]
(1, 2, 1, 2, 1, 0, 2, 1, 0) [0.5 0.5]
(1, 2, 1, 2, 1, 0, 2, 1, 2) [1.]
(1, 2, 1, 2, 1, 0, 0, 2, 0) [0.33333333 0.33333333 0.33333333]
(1, 2, 1, 2, 1, 1, 0

In [3]:
def stateOfBoard(board):
    # Möjliga vinstlinjer i TicTacToe
    winning_lines = [
        (0, 1, 2), (3, 4, 5), (6, 7, 8),  # rader
        (0, 3, 6), (1, 4, 7), (2, 5, 8),  # kolumner
        (0, 4, 8), (2, 4, 6)            # diagonaler
    ]
    
    # Kontrollera om det finns en vinnare
    for a, b, c in winning_lines:
        if board[a] == board[b] == board[c] != 0:
            return board[a]  # 1 om O vunnit, 2 om X vunnit

    # Kontrollera om det finns några tomma platser kvar
    if 0 in board:
        return -1  # Spelet är inte klart ännu

    # Om ingen vinnare och inga tomma platser är kvar, oavgjort
    return 0


In [4]:
def showBoard(board):
    S = [".", "O", "X"]
    print(S[board[0]], S[board[1]], S[board[2]])
    print(S[board[3]], S[board[4]], S[board[5]])
    print(S[board[6]], S[board[7]], S[board[8]])

def makeMove(board, location, player):
    board = list(board)
    board[location] = player
    return tuple(board)


In [5]:
def getEmpty(board):
    empty = []
    for i in range(9):
        if board[i] == 0:
            empty.append(i)
    return empty


In [6]:
# Get function för q värden på brädet
Q = {}

def get_q_values(board):
    if board not in Q:
        Q[board] = np.zeros(9)  # Initialize Q-values av alla actions till 0
    return Q[board]

In [7]:
#Bellman-agentens policy med Q-uppdatering
def Bellman_policy(board, exploration=0.1, reward=None, next_board=None, learning=0.5, discount=0.9):

    empty_spots = getEmpty(board)

    # Utforska eller exploatera
    if np.random.rand() < exploration:  # Utforska
        action = np.random.choice(empty_spots)
    else:
        q_values = get_q_values(board)
        valid_q_values = [q_values[i] for i in empty_spots]
        action = empty_spots[np.argmax(valid_q_values)]

    # Uppdatera Q-tabellen om belöning och nästa tillstånd är tillgängliga
    if reward is not None and next_board is not None:
        max_next_q = max(get_q_values(next_board)) if getEmpty(next_board) else 0
        get_q_values(board)[action] += learning * (reward + discount * max_next_q - get_q_values(board)[action])

    return action

In [8]:
# Spelfunktion
# gamma = discount // factor Epsilon = exploration rate  // Alpha = learning rate
def play(policyA, policyB, visualize=False, exploration=0.1):

    board = (0, 0, 0, 0, 0, 0, 0, 0, 0)  # Starta med ett tomt bräde
    nextPlayer = [0, 2, 1]  # Växla mellan spelare 1 och 2
    player = 1  # Spelare 1 börjar (kan vara Bellman-agenten eller motståndaren)
    history = []  # Spara spelets historik

    while stateOfBoard(board) == -1:  # Spela tills spelet är klart
        locations = getEmpty(board)
        if player == 1:  # Spelare 1:s tur
            action = policyA(board, exploration) if policyA == Bellman_policy else np.random.choice(locations, p=policyA[board])
        else:  # Spelare 2:s tur
            action = policyB(board, exploration) if policyB == Bellman_policy else np.random.choice(locations, p=policyB[board])

        # Spara historik och uppdatera brädet
        next_board = makeMove(board, action, player)
        history.append((board, action, player, next_board))
        board = next_board
        player = nextPlayer[player]  # Växla spelare

        if visualize:  # Visa brädet om visualize=True
            S = [".", "O", "X"]
            print(S[board[0]], S[board[1]], S[board[2]])
            print(S[board[3]], S[board[4]], S[board[5]])
            print(S[board[6]], S[board[7]], S[board[8]])
            print("----")

    outcome = stateOfBoard(board)  # Hämta resultatet av spelet
    # Uppdatera Q-tabellen för Bellman-agentens drag
    for state, action, actor, next_state in reversed(history):  # Iterera baklänges
        if (actor == 1 and policyA == Bellman_policy) or (actor == 2 and policyB == Bellman_policy):
            reward = 1 if outcome == actor else -1 if outcome == nextPlayer[actor] else 0
            Bellman_policy(state, exploration, reward, next_state)  # Uppdatera Q-tabellen
    return outcome, history

In [9]:
collector = [0, 0, 0]
for i in range(10000):
    outcome, history = play(perfectPolicy, randomPolicy)
    collector[outcome] += 1
print(collector)

[413, 9587, 0]


In [10]:
# Träna Perfect vs Bellman perfekt börjar
collector = [0, 0, 0]
for i in range(10000):  # Spela 10 000 matcher
    outcome, history = play(perfectPolicy, Bellman_policy, exploration=0.0)
    collector[outcome] += 1

# Visa resultat
print("Perfect policy vs Bellman-agenten Collector:", collector)

# Bellman börjar mot perfect
collector = [0, 0, 0]
for i in range(10000):  # Spela 10 000 matcher
    outcome, history = play(Bellman_policy, perfectPolicy, exploration=0.0)
    collector[outcome] += 1

# Visa resultat
print("Bellman-agent vs Perfect Policy Collector:", collector)

# Testa Bellman-agenten mot randomPolicy Random börjar
collector_random = [0, 0, 0]
for i in range(10000):  # Spela 10 000 matcher
    outcome, history = play(randomPolicy, Bellman_policy, exploration=0.0)  # Ingen utforskning
    collector_random[outcome] += 1

# Visa resultat
print("Random Policy vs Bellman Policy Collector:", collector_random)

# Bellman börjar mot random
collector_random = [0, 0, 0]
for i in range(10000):  # Spela 10 000 matcher
    outcome, history = play(Bellman_policy, randomPolicy, exploration=0.0)  # Ingen utforskning
    collector_random[outcome] += 1

# Visa resultat
print("Bellman-agenten vs Random Policy Collector:", collector_random)

Perfect policy vs Bellman-agenten Collector: [9989, 11, 0]
Bellman-agent vs Perfect Policy Collector: [9993, 0, 7]
Random Policy vs Bellman Policy Collector: [2154, 1229, 6617]
Bellman-agenten vs Random Policy Collector: [1680, 7692, 628]
